In [1]:
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain.schema.document import Document
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores.chroma import Chroma
from langchain_community.embeddings.ollama import OllamaEmbeddings

In [2]:
chroma_path = "../chroma"

In [3]:
vectordb = Chroma(persist_directory=chroma_path, embedding_function = OllamaEmbeddings(model='nomic-embed-text'))

In [4]:
retriever = vectordb.as_retriever()

In [5]:
URI = "https://repro-toxicity-xrbuc.eastus2.inference.ml.azure.com/score"
KEY = "9aIh4YiafyykA1e9okBCiwz6gupwDskX"

In [6]:
from langchain_community.chat_models.azureml_endpoint import AzureMLChatOnlineEndpoint, CustomOpenAIChatContentFormatter
from langchain_community.llms.azureml_endpoint import AzureMLEndpointApiType

llm = AzureMLChatOnlineEndpoint(
    endpoint_url=URI,
    endpoint_api_type=AzureMLEndpointApiType.dedicated,
    endpoint_api_key=KEY,
    content_formatter=CustomOpenAIChatContentFormatter(),
    model_kwargs={"max_tokens": 512},
)

response = llm.invoke("Will humans ever solve the Collatz conjecture?")
print(response)

content='The eternal question!\n\nThe Collatz Conjecture, also known as the 3x+1 problem, is a famous unsolved problem in mathematics that has been puzzling mathematicians for over 80 years. The conjecture states that for any positive integer, if you apply a simple transformation (either multiply by 3 and add 1, or divide by 2), you eventually reach the number 1.\n\nWhile we have seen evidence of the conjecture holding true for an incredibly large number of starting values, a formal proof or counterexample has yet to be found. In fact, many mathematicians believe that the conjecture is true, but they have been unable to prove it.\n\nSo, will humans ever solve the Collatz Conjecture? It\'s hard to predict, but here are a few reasons why mathematicians remain optimistic:\n\n1. **Slow progress**: Despite the lack of a formal proof, mathematicians have made significant progress in understanding the behavior of the Collatz sequence. For example, we know that the sequence never visits the sa

In [7]:
response.content

'The eternal question!\n\nThe Collatz Conjecture, also known as the 3x+1 problem, is a famous unsolved problem in mathematics that has been puzzling mathematicians for over 80 years. The conjecture states that for any positive integer, if you apply a simple transformation (either multiply by 3 and add 1, or divide by 2), you eventually reach the number 1.\n\nWhile we have seen evidence of the conjecture holding true for an incredibly large number of starting values, a formal proof or counterexample has yet to be found. In fact, many mathematicians believe that the conjecture is true, but they have been unable to prove it.\n\nSo, will humans ever solve the Collatz Conjecture? It\'s hard to predict, but here are a few reasons why mathematicians remain optimistic:\n\n1. **Slow progress**: Despite the lack of a formal proof, mathematicians have made significant progress in understanding the behavior of the Collatz sequence. For example, we know that the sequence never visits the same value

In [8]:
question = "What happened with  OECD  Working  Group in  Copenhagen  in  June  1995?"

In [9]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context. 
If the context doesn't contain any relevant information to the question, don't make something up and just say "I don't know":

<context>
{context}
</context>
"""

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(llm, question_answering_prompt)

In [10]:
from typing import Dict

from langchain_core.runnables import RunnablePassthrough

def parse_retriever_input(params: Dict):
    return params["messages"][-1].content

retrieval_chain = RunnablePassthrough.assign(
    context=parse_retriever_input | retriever,
).assign(
    answer=document_chain,
)

In [11]:
from langchain_core.messages import AIMessage, HumanMessage

query_transform_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),
        (
            "user",
            "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation. Only respond with the query, nothing else.",
        ),
    ]
)

query_transformation_chain = query_transform_prompt | llm

query_transformation_chain.invoke(
    {
        "messages": [
            HumanMessage(content="Can LangSmith help test my LLM applications?"),
            AIMessage(
                content="Yes, LangSmith can help test and evaluate your LLM applications. It allows you to quickly edit examples and add them to datasets to expand the surface area of your evaluation sets or to fine-tune a model for improved quality or reduced costs. Additionally, LangSmith can be used to monitor your application, log all traces, visualize latency and token usage statistics, and troubleshoot specific issues as they arise."
            ),
            HumanMessage(content="Tell me more!"),
        ],
    }
)

BaseMessage(content='"LangSmith LLM application testing"', type='assistant', id='run-a9239247-1df9-4ef4-b93c-4efd20cc1575-0')

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

query_transforming_retriever_chain = RunnableBranch(
    (
        lambda x: len(x.get("messages", [])) == 1,
        # If only one message, then we just pass that message's content to retriever
        (lambda x: x["messages"][-1].content) | retriever,
    ),
    # If messages, then we pass inputs to LLM chain to transform the query, then pass to retriever
    query_transform_prompt | llm | StrOutputParser() | retriever,
).with_config(run_name="chat_retriever_chain")

In [13]:
SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context. 
If the context doesn't contain any relevant information to the question, don't make something up and just say "I don't know":

<context>
{context}
</context>
"""

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(llm, question_answering_prompt)

conversational_retrieval_chain = RunnablePassthrough.assign(
    context=query_transforming_retriever_chain,
).assign(
    answer=document_chain,
)

In [14]:
response = conversational_retrieval_chain.invoke(
    {
        "messages": [
            HumanMessage(content=question),
        ]
    }
)

In [15]:
response

{'messages': [HumanMessage(content='What happened with  OECD  Working  Group in  Copenhagen  in  June  1995?')],
 'context': [Document(page_content='In Copenhagen in June 1995, an OECD Working Group on Reproduction and Developmental', metadata={'id': '../documents/OECD TG 416 (2001).pdf:0:2', 'page': 0, 'source': '../documents/OECD TG 416 (2001).pdf'}),
  Document(page_content='416 OECD/OCDE\n4/13Limit test', metadata={'id': '../documents/OECD TG 416 (2001).pdf:3:0', 'page': 3, 'source': '../documents/OECD TG 416 (2001).pdf'}),
  Document(page_content='416 OECD/OCDE\n8/13Gross necropsy', metadata={'id': '../documents/OECD TG 416 (2001).pdf:7:0', 'page': 7, 'source': '../documents/OECD TG 416 (2001).pdf'}),
  Document(page_content='416 OECD/OCDE\n6/13OBSERVATIONS\nClinical observations', metadata={'id': '../documents/OECD TG 416 (2001).pdf:5:0', 'page': 5, 'source': '../documents/OECD TG 416 (2001).pdf'})],
 'answer': 'According to the context, the OECD Working Group on Reproduction and

In [16]:
response['answer']

'According to the context, the OECD Working Group on Reproduction and Developmental was present in Copenhagen in June 1995.'